In [1]:
import tensorflow.keras as keras
from keras import layers


In [2]:
class ResBlock(keras.Model):
    def __init__(self, filters, downsample):
        super().__init__()
        if downsample:
            self.conv1 = layers.Conv2D(
                filters, kernel_size=3, strides=2, padding='same')
            self.shortcut = keras.Sequential([
                layers.Conv2D(filters, 1, strides=2, padding='same'),
                layers.BatchNormalization()
            ])
        else:
            self.conv1 = layers.Conv2D(
                filters, kernel_size=3, strides=1, padding='same')
            self.shortcut = keras.Sequential()

        self.conv2 = layers.Conv2D(
            filters, kernel_size=3, strides=1, padding='same')

    def call(self, input):
        shortcut = self.shortcut(input)

        input = self.conv1(input)
        input = layers.BatchNormalization()(input)
        input = layers.ReLU()(input)

        input = self.conv2(input)
        input = layers.BatchNormalization()(input)
        input = layers.ReLU()(input)

        input = input + shortcut
        return layers.ReLU()(input)

    def get_config(self):
        return super().get_config()


In [3]:
class ResBottleneckBlock(keras.Model):
    def __init__(self, filters, downsample):
        super().__init__()
        self.downsample = downsample
        self.filters = filters
        self.conv1 = layers.Conv2D(filters, 1, 1, padding='same')
        if downsample:
            self.conv2 = layers.Conv2D(filters, 3, 2, padding='same')
        else:
            self.conv2 = layers.Conv2D(filters, 3, 1, padding='same')
        self.conv3 = layers.Conv2D(filters*4, 1, 1, padding='same')

    def build(self, input_shape):
        if self.downsample or self.filters * 4 != input_shape[-1]:
            self.shortcut = keras.Sequential([
                layers.Conv2D(
                    self.filters*4, 1, 2 if self.downsample else 1, padding='same'),
                layers.BatchNormalization()
            ])
        else:
            self.shortcut = keras.Sequential()

    def call(self, input):
        shortcut = self.shortcut(input)

        input = self.conv1(input)
        input = layers.BatchNormalization()(input)
        input = layers.ReLU()(input)

        input = self.conv2(input)
        input = layers.BatchNormalization()(input)
        input = layers.ReLU()(input)

        input = self.conv3(input)
        input = layers.BatchNormalization()(input)
        input = layers.ReLU()(input)

        input = input + shortcut
        return layers.ReLU()(input)

    def get_config(self):
        return super().get_config()


In [4]:
class ResNet(keras.Model):
    def __init__(self, resblock, filters, outputs=1000):
        super().__init__()
        self.layer0 = keras.Sequential([
            layers.Conv2D(64, 7, 2, padding='same', use_bias=False,),
            layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU()
        ], name='layer0')

        self.layer1 = keras.Sequential([
            resblock(64, downsample=False) for _ in range(filters[0])
        ], name='layer1')

        self.layer2 = keras.Sequential([
            resblock(128, downsample=True)
        ] + [
            resblock(128, downsample=False) for _ in range(1, filters[1])
        ], name='layer2')

        self.layer3 = keras.Sequential([
            resblock(256, downsample=True)
        ] + [
            resblock(256, downsample=False) for _ in range(1, filters[2])
        ], name='layer3')

        self.layer4 = keras.Sequential([
            resblock(512, downsample=True)
        ] + [
            resblock(512, downsample=False) for _ in range(1, filters[3])
        ], name='layer4')

        self.gap = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(outputs, activation='softmax')

    def call(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.layer4(input)
        input = self.gap(input)
        input = self.fc(input)

        return input

    def get_config(self):
        return super().get_config()


In [5]:
class ResNet18(ResNet):
    def __init__(self, outputs=1000):
        super().__init__(ResBlock, [2, 2, 2, 2], outputs)

    def call(self, input):
        return super().call(input)

    def model(self, input_shape):
        x = keras.Input(input_shape)
        return keras.models.Model(x, self.call(x))


class ResNet34(ResNet):
    def __init__(self, outputs=1000):
        super().__init__(ResBlock, [3, 4, 6, 3], outputs)

    def call(self, input):
        return super().call(input)

    def model(self, input_shape):
        x = keras.Input(input_shape)
        return keras.models.Model(x, self.call(x))


class ResNet50(ResNet):
    def __init__(self, outputs=1000):
        super().__init__(ResBottleneckBlock, [3, 4, 6, 3], outputs)

    def call(self, input):
        return super().call(input)

    def model(self, input_shape):
        x = keras.Input(input_shape)
        return keras.models.Model(x, self.call(x))


class ResNet101(ResNet):
    def __init__(self, outputs=1000):
        super().__init__(ResBottleneckBlock, [3, 4, 23, 3], outputs)

    def call(self, input):
        return super().call(input)

    def model(self, input_shape):
        x = keras.Input(input_shape)
        return keras.models.Model(x, self.call(x))


class ResNet152(ResNet):
    def __init__(self, outputs=1000):
        super().__init__(ResBottleneckBlock, [3, 8, 36, 3], outputs)

    def call(self, input):
        return super().call(input)

    def model(self, input_shape):
        x = keras.Input(input_shape)
        return keras.models.Model(x, self.call(x))


In [6]:
resnet18 = ResNet18()
resnet18 = resnet18.model(input_shape=(224, 224, 3))
resnet18.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
layer0 (Sequential)          (None, 56, 56, 64)        9664      
_________________________________________________________________
layer1 (Sequential)          (None, 56, 56, 64)        147712    
_________________________________________________________________
layer2 (Sequential)          (None, 28, 28, 128)       525440    
_________________________________________________________________
layer3 (Sequential)          (None, 14, 14, 256)       2099456   
_________________________________________________________________
layer4 (Sequential)          (None, 7, 7, 512)         8393216   
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0     

In [7]:
resnet34 = ResNet34()
resnet34 = resnet34.model(input_shape=(224, 224, 3))
resnet34.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
layer0 (Sequential)          (None, 56, 56, 64)        9664      
_________________________________________________________________
layer1 (Sequential)          (None, 56, 56, 64)        221568    
_________________________________________________________________
layer2 (Sequential)          (None, 28, 28, 128)       1115776   
_________________________________________________________________
layer3 (Sequential)          (None, 14, 14, 256)       6820096   
_________________________________________________________________
layer4 (Sequential)          (None, 7, 7, 512)         13112832  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0   

In [8]:
resnet50 = ResNet50()
resnet50 = resnet50.model(input_shape = (224, 224, 3))
resnet50.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
layer0 (Sequential)          (None, 56, 56, 64)        9664      
_________________________________________________________________
layer1 (Sequential)          (None, 56, 56, 256)       215424    
_________________________________________________________________
layer2 (Sequential)          (None, 28, 28, 512)       1218048   
_________________________________________________________________
layer3 (Sequential)          (None, 14, 14, 1024)      7092224   
_________________________________________________________________
layer4 (Sequential)          (None, 7, 7, 2048)        14961664  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0   

In [9]:
resnet101 = ResNet101()
resnet101 = resnet101.model(input_shape=(224, 224, 3))
resnet101.summary()


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
layer0 (Sequential)          (None, 56, 56, 64)        9664      
_________________________________________________________________
layer1 (Sequential)          (None, 56, 56, 256)       215424    
_________________________________________________________________
layer2 (Sequential)          (None, 28, 28, 512)       1218048   
_________________________________________________________________
layer3 (Sequential)          (None, 14, 14, 1024)      26058240  
_________________________________________________________________
layer4 (Sequential)          (None, 7, 7, 2048)        14961664  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0   

In [10]:
resnet152 = ResNet152()
resnet152 = resnet152.model(input_shape=(224, 224, 3))
resnet152.summary()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
layer0 (Sequential)          (None, 56, 56, 64)        9664      
_________________________________________________________________
layer1 (Sequential)          (None, 56, 56, 256)       215424    
_________________________________________________________________
layer2 (Sequential)          (None, 28, 28, 512)       2335232   
_________________________________________________________________
layer3 (Sequential)          (None, 14, 14, 1024)      40561664  
_________________________________________________________________
layer4 (Sequential)          (None, 7, 7, 2048)        14961664  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0   